# Second filter: select the final location of the company 📍📍

<img width=400 src=https://media.giphy.com/media/f5IqYTxd7OlY9XHBvw/giphy.gif>

### Importing libraries and dataframes and creating connection with Mongo DB

In [1]:
import pandas as pd
import requests
import json
from pymongo import MongoClient
import geopandas
conn = MongoClient("localhost:27017")
db = conn.get_database("ironhack")
from pymongo import GEOSPHERE
import src.geoqueries as g
import src.maps as m
import src.cleaning as c
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
from keplergl import KeplerGl

In [2]:
df = pd.read_csv("data/df.csv")

#### Dataframe cleaning

In [5]:
df = c.preliminary_cleaning(df)

In [6]:
df.sample(3)

,company,city,latitude,longitude
37,Make It Rain,London,51.462600,-0.139082
49,ACT Biotech,San Francisco,37.787131,-122.404107
40,Granicus,San Francisco,37.787445,-122.397847


In [7]:
coordinates = c.column_point(df)
df['geometry'] = coordinates

In [8]:
df.head()

,company,city,latitude,longitude,geometry
0,Digg,San Francisco,37.764726,-122.394523,"{'type': 'Point', 'coordinates': [-122.394523,..."
1,Powerset,San Francisco,37.778613,-122.395289,"{'type': 'Point', 'coordinates': [-122.395289,..."
2,CastTV,San Francisco,37.780716,-122.393913,"{'type': 'Point', 'coordinates': [-122.393913,..."
3,Grockit,San Francisco,37.775196,-122.419204,"{'type': 'Point', 'coordinates': [-122.419204,..."
4,Dropbox,San Francisco,37.790943,-122.408499,"{'type': 'Point', 'coordinates': [-122.4084994..."


#### Creating geosphere index in MongoDB

In [9]:
db.preferences.create_index([("geometry", GEOSPHERE)])

'geometry_2dsphere'

In [10]:
collection = db.get_collection("preferences")

In [11]:
collection.find_one({})

{'_id': ObjectId('607bfae4b913636b15a4fb88'),
 'name': 'Madison Square Garden',
 'location': {'type': 'Point', 'coordinates': [-73.9935421944, 40.7507519651]},
 'place': 'basket'}

### Adding to the dataframe, for each location, the number of sites that are near it

In [12]:
party_counts = []
starbucks_counts = []
school_counts = []
vegan_counts = []
basket_counts = []
for c in coordinates:
    query = {"location": {"$near": {"$geometry": c, "$maxDistance": 2000}}}
    list_ = list(collection.find(query))
    party = 0
    starbucks = 0
    school = 0
    vegan = 0
    basket = 0
    for l in list_:
        if l['place'] == 'party':
            party += 1
        elif l['place'] == 'starbucks':
            starbucks += 1
        elif l['place'] == 'school':
            school += 1
        elif l['place'] == 'vegan':
            vegan += 1
        elif l['place'] == 'basket':
            basket += 1
    party_counts.append(party)
    starbucks_counts.append(starbucks)
    school_counts.append(school)
    vegan_counts.append(vegan)
    basket_counts.append(basket)

In [13]:
df['party'] = party_counts
df['starbucks'] = starbucks_counts
df['school'] = school_counts
df['vegan'] = vegan_counts
df['basket'] = basket_counts

In [14]:
df.head()

,company,city,latitude,longitude,geometry,party,starbucks,school,vegan,basket
0,Digg,San Francisco,37.764726,-122.394523,"{'type': 'Point', 'coordinates': [-122.394523,...",7,4,0,1,3
1,Powerset,San Francisco,37.778613,-122.395289,"{'type': 'Point', 'coordinates': [-122.395289,...",7,14,2,3,3
2,CastTV,San Francisco,37.780716,-122.393913,"{'type': 'Point', 'coordinates': [-122.393913,...",6,13,0,0,3
3,Grockit,San Francisco,37.775196,-122.419204,"{'type': 'Point', 'coordinates': [-122.419204,...",30,15,28,25,0
4,Dropbox,San Francisco,37.790943,-122.408499,"{'type': 'Point', 'coordinates': [-122.4084994...",8,14,7,5,0


### Giving a weight to each site 

In [15]:
weight_party = 0.25
weight_starbucks = 0.3
weight_basket = 0.05
weight_school = 0.25
weight_vegan = 0.15

### Rating the number of sites from 1 to 4 

**PARTY/STARBUCKS/SCHOOL/VEGAN GRADES**
    
    [0] : 0  
    [1-8] : 1  
    [8-20] : 2
    [20-30] : 3

**BASCKET GRADES**
    
    [0] : 0  
    [2] : 1  
    [3] : 2
    [4] : 3

### Evaluating the number of sites according to their score and weight

In [16]:
df['party_grades'] = df["party"].apply(g.grades)
df['starbucks_grades'] = df["starbucks"].apply(g.grades)
df['school_grades'] = df["school"].apply(g.grades)
df['vegan_grades'] = df["vegan"].apply(g.grades)
df['basket_grades'] = df["basket"]

In [17]:
df['weighted_party_grades'] = df.apply(lambda row: row['party_grades']*weight_party, axis = 1)
df['weighted_starbucks_grades'] = df.apply(lambda row: row['starbucks_grades']*weight_party, axis = 1)
df['weighted_school_grades'] = df.apply(lambda row: row['school_grades']*weight_party, axis = 1)
df['weighted_vegan_grades'] = df.apply(lambda row: row['vegan_grades']*weight_party, axis = 1)
df['weighted_basket_grades'] = df.apply(lambda row: row['basket_grades']*weight_party, axis = 1)

In [18]:
df['weighted_result'] =  df.apply(lambda row: row['weighted_party_grades'] 
                                          + row['weighted_basket_grades']
                                          + row['weighted_starbucks_grades']
                                          + row['weighted_school_grades'] 
                                          + row['weighted_vegan_grades'],
                                          axis = 1)

In [19]:
df.head()

,company,city,latitude,longitude,geometry,party,starbucks,school,vegan,basket,...,starbucks_grades,school_grades,vegan_grades,basket_grades,weighted_party_grades,weighted_starbucks_grades,weighted_school_grades,weighted_vegan_grades,weighted_basket_grades,weighted_result
0,Digg,San Francisco,37.764726,-122.394523,"{'type': 'Point', 'coordinates': [-122.394523,...",7,4,0,1,3,...,1,0,1,3,0.25,0.25,0.00,0.25,0.75,1.50
1,Powerset,San Francisco,37.778613,-122.395289,"{'type': 'Point', 'coordinates': [-122.395289,...",7,14,2,3,3,...,2,1,1,3,0.25,0.50,0.25,0.25,0.75,2.00
2,CastTV,San Francisco,37.780716,-122.393913,"{'type': 'Point', 'coordinates': [-122.393913,...",6,13,0,0,3,...,2,0,0,3,0.25,0.50,0.00,0.00,0.75,1.50
3,Grockit,San Francisco,37.775196,-122.419204,"{'type': 'Point', 'coordinates': [-122.419204,...",30,15,28,25,0,...,2,3,3,0,0.75,0.50,0.75,0.75,0.00,2.75
4,Dropbox,San Francisco,37.790943,-122.408499,"{'type': 'Point', 'coordinates': [-122.4084994...",8,14,7,5,0,...,2,1,1,0,0.25,0.50,0.25,0.25,0.00,1.25


### Selecting the best one based on that criteria

In [20]:
df.weighted_result.max()

3.0

In [21]:
df_ = [df['weighted_result'] == df.weighted_result.max()]

In [22]:
is_max = df.loc[:, "weighted_result"] == df.weighted_result.max()
df_final = df.loc[is_max]

In [23]:
df_final

,company,city,latitude,longitude,geometry,party,starbucks,school,vegan,basket,...,starbucks_grades,school_grades,vegan_grades,basket_grades,weighted_party_grades,weighted_starbucks_grades,weighted_school_grades,weighted_vegan_grades,weighted_basket_grades,weighted_result
30,Box UK,London,51.510774,-0.139245,"{'type': 'Point', 'coordinates': [-0.1392447, ...",27,23,24,26,0,...,3,3,3,0,0.75,0.75,0.75,0.75,0.0,3.0


In [24]:
coordinates_ = [list(df_final['latitude'])[0],list(df_final['longitude'])[0]]

In [25]:
coordinates_

[51.5107737, -0.1392447]

### Exporting dataframe

In [26]:
df_final.to_csv('data/df_final.csv')

## 🏆🏆 The coordinates of the selected location: `[51.5107737, -0.1392447]`🏆🏆

### Let's see on a map everything that is close to that location 👀

In [27]:
london = {'type': 'Point', 'coordinates': [51.509865, -0.118092]}
map_lnd = Map(location=london['coordinates'],zoom_start=15)

#### First we locate the gaming company

In [28]:
m.final_location(coordinates_,map_lnd)
#map_lnd

#### Now we create a new dataframe with the sites that are nearby 

In [29]:
party_coord = []
starbucks_coord = []
school_coord = []
vegan_coord = []
basket_coord = []

query = {"location": {"$near": {"$geometry": list(df_final['geometry'])[0], "$maxDistance": 2000}}}
query

{'location': {'$near': {'$geometry': {'type': 'Point',
    'coordinates': [-0.1392447, 51.5107737]},
   '$maxDistance': 2000}}}

In [30]:
df_places = pd.DataFrame(list(collection.find(query)))

In [31]:
coordinates = []
for point in list(df_places['location']):
    coordinates.append([point['coordinates'][1],point['coordinates'][0]])
df_places['coordinates'] = coordinates

In [32]:
df_places.head()

,_id,name,location,place,coordinates
0,607bfb26b913636b15a4fc4c,Starbucks,"{'type': 'Point', 'coordinates': [-0.139154, 5...",starbucks,"[51.510288, -0.139154]"
1,607bfaf6b913636b15a4fba5,The Cuckoo Club,"{'type': 'Point', 'coordinates': [-0.137770716...",party,"[51.5099619985, -0.1377707164]"
2,607bfaf6b913636b15a4fb98,Tramp,"{'type': 'Point', 'coordinates': [-0.137835548...",party,"[51.5083318933, -0.1378355488]"
3,607bfb0db913636b15a4fc06,Malvern House Picadilly Centre,"{'type': 'Point', 'coordinates': [-0.134370783...",school,"[51.5101933333, -0.1343707833]"
4,607bfb26b913636b15a4fc4f,Starbucks,"{'type': 'Point', 'coordinates': [-0.139378, 5...",starbucks,"[51.513952, -0.139378]"


#### Now, we locate everything that is near the gaming company

In [33]:
m.nearby(df_places,map_lnd)
map_lnd

In [34]:
map_lnd.save('maps/final.html')

### Let's do a Kepler map! 

In [35]:
df_places.drop(['_id'], axis = 1, inplace = True)

In [36]:
df_places.head()

,name,location,place,coordinates
0,Starbucks,"{'type': 'Point', 'coordinates': [-0.139154, 5...",starbucks,"[51.510288, -0.139154]"
1,The Cuckoo Club,"{'type': 'Point', 'coordinates': [-0.137770716...",party,"[51.5099619985, -0.1377707164]"
2,Tramp,"{'type': 'Point', 'coordinates': [-0.137835548...",party,"[51.5083318933, -0.1378355488]"
3,Malvern House Picadilly Centre,"{'type': 'Point', 'coordinates': [-0.134370783...",school,"[51.5101933333, -0.1343707833]"
4,Starbucks,"{'type': 'Point', 'coordinates': [-0.139378, 5...",starbucks,"[51.513952, -0.139378]"


In [37]:
list(df_final.geometry)[0]

{'type': 'Point', 'coordinates': [-0.1392447, 51.5107737]}

In [38]:
df_final.head()
new_row = {'name':'Gaming Company', 'location': list(df_final.geometry)[0], 'place' : 'company', 'coordinates' : [list(df_final.geometry)[0]['coordinates'][1], list(df_final.geometry)[0]['coordinates'][0]]}
df_places = df_places.append(new_row, ignore_index=True)

In [39]:
df_places.tail()

,name,location,place,coordinates
96,Academy Class,"{'type': 'Point', 'coordinates': [-0.116223596...",school,"[51.5028805129, -0.1162235969]"
97,Starbucks,"{'type': 'Point', 'coordinates': [-0.123353, 5...",starbucks,"[51.52392, -0.123353]"
98,London Waterloo Academy,"{'type': 'Point', 'coordinates': [-0.115453, 5...",school,"[51.503339, -0.115453]"
99,King's College London - Waterloo Campus,"{'type': 'Point', 'coordinates': [-0.112166203...",school,"[51.5055975107, -0.1121662031]"
100,Gaming Company,"{'type': 'Point', 'coordinates': [-0.1392447, ...",company,"[51.5107737, -0.1392447]"


In [40]:
latit = []
longit = []
for coord in list(df_places['coordinates']):
    latit.append(coord[0])
    longit.append(coord[1])

In [41]:
df_places['latitude'] = latit
df_places['longitude'] = longit

In [42]:
map_kepler= KeplerGl(height=700, weight = 500)
map_kepler.add_data(data=df_places, name='companies')
map_kepler

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'companies': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 2…

In [43]:
map_kepler.save_to_html(file_name = "maps/final_kepler.html", read_only = True)

Map saved to maps/final_kepler.html!
